In [9]:
import numpy as np
import pandas as pd
import os

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian

In [25]:
def load_atlas(parcellation_dir):
    """
    Loads Desikan-Killiany atlas parcellation files for different spaces and resolutions.

    Parameters
    ----------
    parcellation_dir : str
        Directory containing the atlas parcellation files.

    Returns
    -------
    atlas : dict
        Dictionary with keys for each atlas version and values as file paths.
    """
    
    atlas = {
        'dk_fsaverage_10k': (
            os.path.join(parcellation_dir, 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz'),
            os.path.join(parcellation_dir, 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
        ),
        'dk_fsaverage_164k': (
            os.path.join(parcellation_dir, 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot'),
            os.path.join(parcellation_dir, 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
        ),
        'dk_mni': os.path.join(parcellation_dir, 'atlas-desikankilliany_space-MNI_res-1mm.nii.gz')
    }
    return atlas



In [26]:
def create_parcellaters(atlas):
    """
    Creates Parcellater objects for different atlas versions.
    Parameters
    ----------
    atlas : dict
        Dictionary with keys for each atlas version and values as file paths.
    Returns
    -------
    parcellater_fs10k : Parcellater
        Parcellater object for fsaverage 10k resolution.
    parcellater_fs164k : Parcellater
        Parcellater object for fsaverage 164k resolution.
    parcellater_mni : Parcellater
        Parcellater object for MNI152 space.
    """

    dk_fsaverage_10k = relabel_gifti(atlas['dk_fsaverage_10k'])
    parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
    
    dk_fsaverage_164k = annot_to_gifti(atlas['dk_fsaverage_164k'])  
    parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')

    parcellater_mni = Parcellater(atlas['dk_mni'], 'MNI152')
    return (parcellater_fs10k, parcellater_fs164k, parcellater_mni)



In [28]:
    
# load atlas & make parcellaters 
base_path = os.path.dirname(os.getcwd())
parcellations = base_path + '/atlas'
atlas = load_atlas(parcellations)
parcellater_fs10k, parcellater_fs164k, parcellater_mni = create_parcellaters(atlas)

In [22]:
from neuromaps.datasets import available_annotations
for annotation in available_annotations():
    print(annotation)

('abagen', 'genepc1', 'fsaverage', '10k')
('aghourian2017', 'feobv', 'MNI152', '1mm')
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')
('bedard2019', 'feobv', 'MNI152', '1mm')
('beliveau2017', 'az10419369', 'MNI152', '1mm')
('beliveau2017', 'az10419369', 'fsaverage', '164k')
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
('beliveau2017', 'cimbi36', 'fsaverage', '164k')
('beliveau2017', 'cumi101', 'MNI152', '1mm')
('beliveau2017', 'cumi101', 'fsaverage', '164k')
('beliveau2017', 'dasb', 'MNI152', '1mm')
('beliveau2017', 'dasb', 'fsaverage', '164k')
('beliveau2017', 'sb207145', 'MNI152', '1mm')
('beliveau2017', 'sb207145', 'fsaverage', '164k')
('ding2010', 'mrb', 'MNI152', '1mm')
('dubois2015', 'abp688', 'MNI152', '1mm')
('dukart2018', 'flumazenil', 'MNI152', '3mm')
('dukart2018', 'fpcit', 'MNI152', '3mm')
('fazio2016', 'madam', 'MNI152', '3mm')
('finnema2016', 'ucbj', 'MNI152', '1mm')
('gallezot2010', 'p943', 'MNI152', '1mm')
('gallezot2017', 'gsk189254', 'MNI152', '1mm')
('hcps1200', 'm

In [37]:
from neuromaps.datasets import fetch_annotation
annotation_abp688 = fetch_annotation(source='rosaneto')
annotation_ding2010 = fetch_annotation(source='ding2010')

print(annotation_abp688)
print(annotation_ding2010)

#
abp688 = parcellater_mni.fit_transform(annotation_abp688, space='MNI152', ignore_background_data=True)
ding2010 = parcellater_mni.fit_transform(annotation_ding2010, space='MNI152', ignore_background_data=True)



/Users/laurituominen/neuromaps-data/annotations/rosaneto/abp688/MNI152/source-rosaneto_desc-abp688_space-MNI152_res-1mm_feature.nii.gz
/Users/laurituominen/neuromaps-data/annotations/ding2010/mrb/MNI152/source-ding2010_desc-mrb_space-MNI152_res-1mm_feature.nii.gz


In [51]:
abp688 

array([3.60145084, 3.64495495, 3.42284499, 3.4405554 , 3.1326523 ,
       3.44586224, 3.71973693, 3.58844295, 3.31569584, 3.50664052,
       3.56744482, 3.32251093, 3.84708784, 3.55485851, 3.11194808,
       3.27652946, 3.6213047 , 3.46154366, 3.62216346, 3.17265261,
       3.23585415, 3.69018015, 3.29084801, 3.48265317, 3.70497475,
       3.5432495 , 3.46669872, 3.24612216, 3.34366399, 3.57372885,
       3.40410555, 3.01197333, 3.585942  , 3.64554829, 3.56978519,
       3.53875889, 3.44809195, 3.33727809, 3.00410309, 3.35231664,
       3.61467389, 3.59437888, 3.47936272, 3.29543888, 3.45207152,
       3.11740376, 3.78317047, 3.77402155, 2.99699301, 3.22795503,
       3.67686766, 3.55995334, 3.78023778, 3.3397421 , 3.32157765,
       3.76758278, 3.3179771 , 3.52640412, 3.7702974 , 3.73666129,
       3.42367965, 3.11283048, 3.52241534, 3.68366481, 3.59484205,
       3.29715655, 3.5297364 , 3.53414924])

In [40]:
from scipy.stats import pearsonr
r, p = pearsonr(abp688[0], ding2010[0])

print(f'Correlation between ABP688 and Ding2010: r = {r:.3f}, p = {p:.3f}')


Correlation between ABP688 and Ding2010: r = 0.140, p = 0.207


In [46]:
rois = pd.read_csv(os.path.join(base_path, 'atlas' ,'atlas-desikankilliany.csv'))
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()
#abp688 = abp688[0][rois]
#ding2010 = ding2010[0][rois]
r, p = pearsonr(abp688, ding2010)
print(f'Correlation between ABP688 and Ding2010: r = {r:.3f}, p = {p:.3f}')


Correlation between ABP688 and Ding2010: r = 0.264, p = 0.030


In [48]:
# get spins 
spins = pd.read_csv(os.path.join(base_path, 'atlas', 
                                 'spins_hungarian_aparc+aseg_ctx.csv'), header=None)
nspins = spins.values.shape[1]

In [50]:

# empirical correlation between annotations and parc
rho = pearsonr(abp688, ding2010)[0]

# get 10k rotations 
rotated = hungarian(data=abp688, n_perm=10000, spins=spins, parcellation=atlas['dk_mni']) 

# get null 
n = np.zeros((nspins, ))
for i in range(nspins):
    n[i] = pearsonr(ding2010, rotated[:,i])[0]    

# get p-value
pspin = (1 + sum(abs(n) > abs(rho ))) / (nspins + 1)

# store, multiply by -1 to make more intuitive, because smaller p-value/rho means bigger effect  
print(f'Spatially permuted p-value: {pspin:.3f}, rho: {rho:.3f}')



Spatially permuted p-value: 0.045, rho: 0.264
